In [1]:
# 사용하는 최상위 폴더를 변수로 하나 선언
ROOT_FOLDER = '/root/jupyter_home/TF2_Object_Detection'

In [2]:
!mkdir $ROOT_FOLDER

In [4]:
cd $ROOT_FOLDER

/root/jupyter_home/TF2_Object_Detection


In [5]:
# Tensorflow는 Object Detection에 관련된 여러 API를 github repository로 제공!
# 우리는 repository를 clone해서 사용할거에요!
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 99863, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 99863 (delta 33), reused 31 (delta 17), pack-reused 99809 (from 2)
Receiving objects: 100% (99863/99863), 625.36 MiB | 2.78 MiB/s, done.
Resolving deltas: 100% (71934/71934), done.


In [6]:
# 다운로드가 끝나면 models 폴더가 생성되요!
# models 안에 research 폴더가 있어요!
# 이 models/research 폴더로 working directory를 이동시켜요!

In [7]:
cd $ROOT_FOLDER/models/research

/root/jupyter_home/TF2_Object_Detection/models/research


In [8]:
# 오늘은 데이터를 TFRecord로 만들어야 해요!
# tensorflow의 기본 데이터 format.
# TFRecord는 Example로 구성되어있고 Example은 Features로 되어 있어요!
# Features는 여러개의 Feature들의 집합이에요!
# 결국 이런 Feature들을 이진 직렬화 시켜서 파일에 저장하는 구조
# 직렬화 방법은 Protocol Buffer를 사용해요!
# .proto 파일(뼈대 파일)을 컴파일해서 .py를 생성해야 코드에서 사용할 수 있어요!

In [9]:
# 현재 working directory는 models/reasearch
!ls object_detection/protos

__init__.py			      keypoint_box_coder.proto
anchor_generator.proto		      losses.proto
argmax_matcher.proto		      matcher.proto
bipartite_matcher.proto		      mean_stddev_box_coder.proto
box_coder.proto			      model.proto
box_predictor.proto		      multiscale_anchor_generator.proto
calibration.proto		      optimizer.proto
center_net.proto		      pipeline.proto
eval.proto			      post_processing.proto
faster_rcnn.proto		      preprocessor.proto
faster_rcnn_box_coder.proto	      region_similarity_calculator.proto
flexible_grid_anchor_generator.proto  square_box_coder.proto
fpn.proto			      ssd.proto
graph_rewriter.proto		      ssd_anchor_generator.proto
grid_anchor_generator.proto	      string_int_label_map.proto
hyperparams.proto		      target_assigner.proto
image_resizer.proto		      train.proto
input_reader.proto


In [12]:
# 위에 있는 .proto 파일들을 컴파일해야 해요!
# protoc라는 compiler를 이용해서 컴파일하면 되요!
# 우리는 protoc 버전을 3.20버전을 사용할거에요!
# 다른 버전 사용하면 오류 발생!
!protoc --version

libprotoc 3.20.3


In [13]:
# 컴파일을 해보아요!
!protoc object_detection/protos/*.proto --python_out=.

In [14]:
!ls object_detection/protos

__init__.py			       keypoint_box_coder.proto
anchor_generator.proto		       keypoint_box_coder_pb2.py
anchor_generator_pb2.py		       losses.proto
argmax_matcher.proto		       losses_pb2.py
argmax_matcher_pb2.py		       matcher.proto
bipartite_matcher.proto		       matcher_pb2.py
bipartite_matcher_pb2.py	       mean_stddev_box_coder.proto
box_coder.proto			       mean_stddev_box_coder_pb2.py
box_coder_pb2.py		       model.proto
box_predictor.proto		       model_pb2.py
box_predictor_pb2.py		       multiscale_anchor_generator.proto
calibration.proto		       multiscale_anchor_generator_pb2.py
calibration_pb2.py		       optimizer.proto
center_net.proto		       optimizer_pb2.py
center_net_pb2.py		       pipeline.proto
eval.proto			       pipeline_pb2.py
eval_pb2.py			       post_processing.proto
faster_rcnn.proto		       post_processing_pb2.py
faster_rcnn_box_coder.proto	       preprocessor.proto
faster_rcnn_box_coder_pb2.py	       preprocessor_pb2.py
faster_rcnn_pb2.py		       region_simi

In [15]:
# 이제 1차 준비작업은 됐고
# 필요한 package를 추가로 설치해야 해요!
# 현재 필요한 모든 기능이 다 갖춰져 있는 상태가 아니에요!
# 다운로드 받아서 준비를 해야해요!
# 수동으로 하면 너무 힘들어요!
# tensorflow에서 자동으로 package를 다운로드 할 수 있는 script를 제공
!cp $ROOT_FOLDER/models/research/object_detection/packages/tf2/setup.py $ROOT_FOLDER/models/research

In [ ]:
# setup.py를 실행시켜서 필요한 package를 설치해요!
!python -m pip install .

In [18]:
import tensorflow as tf
tf.__version__

'2.19.0'

In [ ]:
# Tensorflow 버전이 변경되었어요.
# 원래 우리가 사용했던 2.13 버전으로 다시 돌아가야해요!
!pip install tensorflow==2.13.0

In [26]:
!pip show tensorflow

Name: tensorflow
Version: 2.13.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /root/anaconda3/envs/tf_env/lib/python3.10/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: tensorflow-text, tf-models-official, tf_keras


In [ ]:
# 학습(재학습)을 할 때 처음부터 하지 않아요!
# 기존에 학습을 해 놓았던 모델(가중치)를 가져다가 
# fine tuning하는 방식으로 학습을 진행!
# pretrained network를 받아서 활용할거에요!
# pretrained network을 다운로드 받을 폴더를 미리 하나 생성

In [2]:
cd $ROOT_FOLDER

/root/jupyter_home/TF2_Object_Detection


In [3]:
# 폴더를 하나 생성해요!
!mkdir $ROOT_FOLDER/pretrained_model

In [4]:
# 준비작업이 끝났으니 이제 TFRecord를 생성해야 해요!
# 학습할 데이터 셋을 준비해야해요
# PASCAL VOC Data Set을 이용해서 TFRecord를 만들거에요!
# 다행히 우리가 TFRecord를 직접 만들지 않아요! 프로그램이 있어요!
# PASCAL VOC 형태의 데이터를 TFRecord로 쉽게 변환할 수 있어요!

In [5]:
cd $ROOT_FOLDER/models/research

/root/jupyter_home/TF2_Object_Detection/models/research


In [6]:
# PASCAL VOC Data Set을 가져올거에요!
!cp /root/jupyter_home/YOLO_Object_Detection/VOC/VOCtrainval_06-Nov-2007.tar .

In [ ]:
!tar xvf VOCtrainval_06-Nov-2007.tar

In [8]:
# 이제 데이터가 준비되었어요!
# 1. class 이름이 명시된 파일을 하나 생성해야되요!
#    .pbtxt 라고해요!
#    나름대로의 형식이 있어요!
#    코드로 만들어 보아요!
voc_classes = [
    "aeroplane", "bicycle", "bird", "boat", "bottle",
    "bus", "car", "cat", "chair", "cow",
    "diningtable", "dog", "horse", "motorbike", "person",
    "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]

with open("label_map.pbtxt", "w") as f:
    for idx, class_name in enumerate(voc_classes, 1):
        f.write("item {\n")
        f.write(f"  id: {idx}\n")
        f.write(f"  name: '{class_name}'\n")
        f.write("}\n\n")

In [9]:
# TFRecord를 생성하는 Python 프로그램도 생겼으니 이제 해당코드를
# 이용해서 PASCAL VOC Data를 TFRecord로 변환시켜 보아요!
!python ./create_pascal_tf_record.py \
    --data_dir=VOCdevkit \
    --year=VOC2007 \
    --set=train \
    --output_path=./pascal_train.tfrecord \
    --label_map_path=./label_map.pbtxt

TFRecord 생성 완료: ./pascal_train.tfrecord


In [10]:
!python ./create_pascal_tf_record.py \
    --data_dir=VOCdevkit \
    --year=VOC2007 \
    --set=val \
    --output_path=./pascal_val.tfrecord \
    --label_map_path=./label_map.pbtxt

TFRecord 생성 완료: ./pascal_val.tfrecord


In [ ]:
# 이렇게 해서 입력데이터 준비가 끝나요!

# 모델생성해서 학습을 진행하면 되요!
# 모델은 pretrained network를 이용할거에요!
# 우리가 사용하는 모델은 faster-RCNN 모델을 이용

# 학습을 하기 위해서 디렉토리 구조를 만들어야 해요!
# TF2_Object_Detention/my_object_detection_model/
#   ㄴ data/ 
#        ㄴ .pbtxt(class)
#        ㄴ 2개의 TFRecord
#   ㄴ models/
#        ㄴ pascal_voc_model/
#             ㄴ my_model.config(설정 파일)

In [12]:
!mkdir $ROOT_FOLDER/my_object_detection_model

In [13]:
!mkdir $ROOT_FOLDER/my_object_detection_model/data
!mkdir $ROOT_FOLDER/my_object_detection_model/models

In [15]:
!mkdir $ROOT_FOLDER/my_object_detection_model/models/pascal_voc_model

In [16]:
# 폴더 구조는 다 생성을 했으니
# 먼저 data폴더에 아까 생성한 pbtxt, tfrecord 파일을 복사해요!
!cp $ROOT_FOLDER/models/research/label_map.pbtxt $ROOT_FOLDER/my_object_detection_model/data
!cp $ROOT_FOLDER/models/research/pascal_train.tfrecord $ROOT_FOLDER/my_object_detection_model/data
!cp $ROOT_FOLDER/models/research/pascal_val.tfrecord $ROOT_FOLDER/my_object_detection_model/data

In [17]:
# 모델을 학습하기 위해 필요한 설정파일을 하나 만들었어요!
# $ROOT_FOLDER/my_object_detection_model/models/pascal_voc_model/my_model.cfg
# 이렇게 만들어놨어요!

In [ ]:
# Pretrained Network을 받아와야 해요!
# 학습을 처음부터 시작할 수는 없어요!

In [18]:
cd $ROOT_FOLDER/pretrained_model

/root/jupyter_home/TF2_Object_Detection/pretrained_model


In [19]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8.tar.gz

--2025-05-09 12:33:46--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8.tar.gz
34.128.9.251, 34.128.9.187, 34.64.4.91, ....tensorflow.org)... 
connected. to download.tensorflow.org (download.tensorflow.org)|34.128.9.251|:80... 
200 OKequest sent, awaiting response... 
Length: 359156023 (343M) [application/x-tar]
Saving to: ‘faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8.tar.gz’

faster_rcnn_resnet1 100%[===================>] 342.52M  9.85MB/s    in 2m 2s   

2025-05-09 12:35:49 (2.81 MB/s) - ‘faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8.tar.gz’ saved [359156023/359156023]



In [20]:
!tar xvf faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8.tar.gz

faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8/
faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8/checkpoint/
faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8/checkpoint/ckpt-0.data-00000-of-00001
faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8/checkpoint/checkpoint
faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8/checkpoint/ckpt-0.index
faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8/pipeline.config
faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8/saved_model/
faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8/saved_model/saved_model.pb
faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8/saved_model/variables/
faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8/saved_model/variables/variables.data-00000-of-00001
faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8/saved_model/variables/variables.index


In [22]:
# 압축이 풀리면 여러개의 폴더와 파일이 나와요!
# 모델이 가지고 있는 checkpoint(가중치)가 필요해요!
# 그 가중치만 특정 폴더에 복사해서 사용할거에요!

# 가중치 정보는 checkpoint라는 폴더 안에 들어가 있어요!
# 이 폴더를 my_object_detection_model 폴더에 복사할 거에요!
!cp -r $ROOT_FOLDER/pretrained_model/faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8/checkpoint $ROOT_FOLDER/my_object_detection_model

In [23]:
# 모델을 학습할 준비가 끝났으니 이제 학습을 진행해보아요!
# tensorflow 버전은 2.13버전을 사용해야 해요.
# 만약 버전이 다르면 오류나요!
# 학습은 $ROOT_FOLDER/models/research 이 폴더 안에서 진행해요!

In [24]:
cd $ROOT_FOLDER/models/research

/root/jupyter_home/TF2_Object_Detection/models/research


In [ ]:
# 학습을 해 보아요!
!python object_detection/model_main_tf2.py \
    --pipeline_config_path=$ROOT_FOLDER/my_object_detection_model/models/pascal_voc_model/my_model.config \
    --model_dir=$ROOT_FOLDER/my_object_detection_model/models/pascal_voc_model \
    --alsologtostderr